In [ ]:
import torch
import numpy as np
import pyarrow.dataset as ds
import json
from sklearn.metrics import mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import MinMaxScaler


# Список входных данных. На вход модели подаются параметры [1-28]
# 00: 'BHP'                         -- Выгрузка VLP. (Целевой параметр)
# 01: 'GOR'                         -- Выгрузка VLP  (Входной параметр из предыдущего узла)
# 02: 'LIQ'                         -- Выгрузка VLP  (Входной параметр из предыдущего узла)
# 03: 'THP'                         -- Выгрузка VLP  (Входной параметр из предыдущего узла)
# 04: 'WCT'                         -- Выгрузка VLP  (Входной параметр из предыдущего узла)
# 05: 'angle'                       -- Угол из Deviation Survey.   (Параметр из json)
#                                      Можно определить через profileHorDistanceMSpaceHeightM в файле network.json
# 06: 'bubble_point_corr_param_1'   -- Bubble Point Parameter 1 из Correlation из PVT Input Data. Задавался 1.0
# 07: 'bubble_point_corr_param_2'   -- Bubble Point Parameter 2 из Correlation из PVT Input Data. Задавался 0.0
# 08: 'correlation'                 -- Тип корреляции VLP. Задавалс 5 (Beggs and Brill)
# 09: 'friction'                    -- Параметр корреляции для Beggs and Brill
# 10: 'gas_gravity'                 -- Gas Gravity из PVT Input Data. Задавался 0.75 (Параметр из json)
# 11: 'gravity'                     -- Параметр корреляции для Beggs and Brill
# 12: 'heat_transfer_coef'          -- Overall Heat Transfer Coefficient из Geothermal Gradient. Задавался 8
# 13: 'inner_temp'                  -- Formation Temperature из Geothermal Gradient на глубине Measured Depth. Задавался 30
# 14: 'measured_depth'              -- Measured depth из deviation survey. По сути длина трубы (Параметр из json)
#                                   -- Можно определить через profileHorDistanceMSpaceHeightM в файле network.json
# 15: 'oil_fvf_corr_param_1'        -- Oil FVF (Below Pb) Parameter 1 из Correlation из PVT Input Data. Задавался 1.0
# 16: 'oil_fvf_corr_param_2'        -- Oil FVF (Below Pb) Parameter 2 из Correlation из PVT Input Data. Задавался 0.0
# 17: 'oil_gravity'                 -- Oil Gravity из PVT Input Data. Задавался 808.47 (Параметр из json)
# 18: 'roughness'                   -- Шероховатость из Downhole Equipment. Задавался 3e-5 (Параметр из json)
# 19: 'sgor'                        -- Solution GOR из PVT Input Data. Задавался 89.27
# 20: 'sgor_corr_param_1'           -- Solution GOR Parameter 1 из Correlation из PVT Input Data. Задавался 1.0
# 21: 'sgor_corr_param_2'           -- Solution GOR Parameter 2 из Correlation из PVT Input Data. Задавался 0.0
# 22: 'surface_temp'                -- Formation Temperature из Geothermal Gradient на 0 м. Задавался 5
# 23: 'top_node_pressure'           -- Top Node Pressure из VLP. Задавлся 11.1  !!! Это же дублирование THP
# 24: 'total_gor'                   -- Total GOR из VLP. Задавался 89.27
# 25: 'tube_diametr'                -- Диаметр трубы из DownHole Equipment (Параметр из json) 
# 26: 'viscocity_corr_param_1'      -- Oil Viscocity Parameter 1 из Correlation из PVT Input Data. Задавался 1.0
# 27: 'viscocity_corr_param_2'      -- Oil Viscocity Parameter 2 из Correlation из PVT Input Data. Задавался 0.0
# 28: 'water_cut'                   -- Water Cut из VLP. Задавался 1


if __name__ == '__main__':

    scaler = MinMaxScaler()


    # В этих файлах лежат макс и мин. значения параметров из списка выше. В них также есть целевой параметр первой строкой!
    # Они нужны для скалрирования параметров и предикта
    min_values = np.loadtxt('min_values.txt')
    max_values = np.loadtxt('max_values.txt')

    with open("network.json") as file_json:
        network = json.load(file_json)

    oil_gravity = network['oilDensityKgToM3']
    gas_gravity = network['gasDensityKgToM3']
    oilrate = network['nodes'][0]['oilRateTonneToD']
    waterrate = network['nodes'][0]['waterRateTonneToD']
    gasrate = network['nodes'][0]['gasRateTonneToD']
    thp = float(network['nodes'][1]['inletPressureAtma']) # По какой-то причине в json он записан в формате str

    diam = network['pipes'][0]['innerDiameterMm'] / 1000 # переводим в метры
    roughness = network['pipes'][0]['roughnessMm'] / 1000 # переводим в метры
    tube_x0, tube_y0 = map(float, network['pipes'][0]['profileHorDistanceMSpaceHeightM'][0].split())
    tube_x1, tube_y1 = map(float, network['pipes'][0]['profileHorDistanceMSpaceHeightM'][1].split())

    tube_dx = tube_x1 - tube_x0
    tube_dy = tube_y1 - tube_y0

    measured_depth = (tube_dx ** 2 + tube_dy ** 2) ** 0.5
    angle = np.rad2deg(np.arccos(tube_dy / measured_depth))

    liqrate = oilrate + waterrate
    wct = waterrate / liqrate
    gor = gasrate / oilrate


    data = {
        "GOR": gor,
        "THP": thp,
        "LIQ": liqrate,
        "WCT": wct,
        "measured_depth": measured_depth,
        "angle": angle,
        "gas_gravity": gas_gravity,
        "oil_gravity": oil_gravity,
        'roughness': roughness,
        'tube_diametr': diam,
        'bubble_point_corr_param_1': 1.0,
        'bubble_point_corr_param_2': 0.0,
        'correlation': 5,
        'friction': 1.0,
        'gravity': 1.0,
        'heat_transfer_coef': 8,
        'inner_temp': 30,
        'oil_fvf_corr_param_1': 1.0,
        'oil_fvf_corr_param_2': 1.0,
        'sgor': 89.27,
        'sgor_corr_param_1': 1.0,
        'sgor_corr_param_2': 1.0,
        'surface_temp': 5,
        'top_node_pressure': 11.1,
        'total_gor': 89.27,
        'viscocity_corr_param_1': 1.0,
        'viscocity_corr_param_2': 0.0,
        'water_cut': 1,
    }

    sorted_keys = sorted(data.keys())

    data_raw = []
    for key in sorted_keys:
        data_raw.append(data[key])

    data_raw = np.array(data_raw)
    # Для упрощения собираем скейлер без BHP
    scaler.fit([min_values[1:], max_values[1:]])
    # Скалируем данные
    x = scaler.transform([data_raw])

    device = torch.device('cuda' if not torch.cuda.is_available() else 'cpu')
    X_batch = torch.tensor(x, device=device, dtype=torch.float)

    model = torch.jit.load("model.pth")
    model.to(device)
    model.eval()
    
    predicted = model(X_batch).detach().numpy()
    # Скалируем предикт обратно
    predicted_backscaled = predicted * (max_values[0] - min_values[0]) + min_values[0]

    print(f"BHP: {predicted_backscaled}")


